In [2]:
import pandas as pd
import preprocessing_tools as pr
from cleanedNames import KeyProc
import os
cn = KeyProc()
%reload_ext autoreload
import warnings
warnings.filterwarnings('ignore')


Порядок работы: 

     1) Обрабатывается текст(препроцессинг, лемматизация) 
     2) Предсказывается рубрика. Сейчас  три рубрики: "Интернет и сми", "Бизнес", "Спорт" 
     3) В зависимости от рубрики, подключается нужная модель BigArtm и предсказывает,  подходящий по смыслу, топик 

In [3]:
dict_rub = {'Интернет и СМИ':'Internet_i_SMI','Бизнес':'Biznes','Спорт':'Sport'}

In [5]:
# Текст  новости с сайта Лента из рубрики "Интернет и сми" 19:39, 26 февраля 2019
text ='''Японский геймдизайнер Хидэо Кодзима опубликовал фотографию водки,чем привлек 
внимание русскоязычных комментаторов. Пост с алкоголем глава студии по разработке компьютерных игр 
выложил в Instagram.Россияне оставили множество комплиментов на странице Кодзимы. Большинство комментариев
содержали слова «гений» и «гениальный», отсылая к мему про геймдизайнера.Юзеры также начали использовать эпитет
«царь» по отношению к основателю игровой студии из-за марки водки на снимке.Один из пользователей с ником rodion_obrien
предложил выдвинуть Кодзиму в президенты. Некоторые комментаторы предположили, что фотографии — это намек на появление
в будущих играх русской озвучки.Накануне, 25 февраля, Кодзима опубликовал в Twitter несколько снимков с подарками от
российского иллюстратора Ильи Кувшинова. Тот посетил студию Kojima Productions и одарил ее сотрудников картинами своего
авторства.В январе 2018 года Кодзима создал аккаунт в 
Instagram, после этого русскоязычные юзеры оставили под публикациями похвалу в адрес геймдизайнера.'''

In [4]:
text = ''' Екатерина Гордон, представитель жены футболиста «Зенита» Ивана Новосельцева Катерины Кейру, рассказала о претензиях клиентки к мужу. Ее слова приводит Sport24.
Гордон заявила, что Кейру пытается урегулировать вопросы имущественного характера. «Господин Новосельцев позволил себе довольно нечестное поведение, которое стало достоянием его супруги. Если вопрос не будет урегулирован, это дело станет погромче распри Глушакова», — добавила она.
Гордон отметила, что пока неясно, проводил Новосельцев манипуляции с недвижимостью сам или по чьему-то совету. Если окажется, что футболист виновен в случившемся, его ждут судебные разбирательства.
Новосельцев стал игроком «Зенита» летом 2016-го. В 2018-м защитник был отдан в аренду сначала тульскому «Арсеналу», затем махачкалинскому «Анжи». До конца текущего сезона он находится на правах аренды в «Ростове».
Кейру и Новосельцев поженились в октябре 2015-го года. В настоящее время супруга футболиста ждет ребенка.'''

In [5]:
# Обработка текста  для предсказания рубрики
text = pr.clean_text(text)
text = cn.replaceKeywords(text)
text = pr.lemmatization(text)
print(text)


['екатерина_гордон', 'жена', 'футболист', 'зенит', 'иван', 'новосельцев', 'катерина', 'кейра', 'претензия', 'клиентка', 'муж', 'приводить', 'sport', 'гордон', 'кейра', 'пытаться', 'урегулировать', 'вопрос', 'имущественный', 'характер', 'господин', 'новосельцев', 'позволить', 'довольно', 'нечестный', 'поведение', 'достояние', 'супруга', 'вопрос', 'урегулировать', 'громкий', 'распря', 'глушаков', 'добавить', 'гордон', 'пока', 'неясно', 'проводить', 'новосельцев', 'манипуляция', 'недвижимость', 'чьемуть', 'совет', 'футболист', 'виновный', 'случиться', 'ждать', 'судебный', 'разбирательство', 'новосельцев', 'игрок', 'зенит', 'летом', 'го', 'защитник', 'отдать', 'аренда', 'сначала', 'тульский', 'арсенал', 'затем', 'махачкалинский', 'анжи', 'конец', 'текущий', 'сезон', 'право', 'аренда', 'ростов', 'кейра', 'новосельцев', 'пожениться', 'го', 'настоящее', 'супруг', 'футболист', 'ждать', 'ребёнок']


In [6]:
# Загружаем модели для предсказания рубрики

PATHMODEL = 'data/model/' 
PATH_BIGARTM_MODEL='data/model/bigartm/'

from sklearn.externals import joblib

vect = joblib.load(PATHMODEL+'vect_file.pkl')
linersvc = joblib.load(PATHMODEL+'model_for_18.pkl')
print('load ok')





load ok


In [7]:
text_pred=[','.join(text)]
print (text_pred)
text_vect_train = vect.transform(text_pred)
pred = linersvc.predict(text_vect_train)


['екатерина_гордон,жена,футболист,зенит,иван,новосельцев,катерина,кейра,претензия,клиентка,муж,приводить,sport,гордон,кейра,пытаться,урегулировать,вопрос,имущественный,характер,господин,новосельцев,позволить,довольно,нечестный,поведение,достояние,супруга,вопрос,урегулировать,громкий,распря,глушаков,добавить,гордон,пока,неясно,проводить,новосельцев,манипуляция,недвижимость,чьемуть,совет,футболист,виновный,случиться,ждать,судебный,разбирательство,новосельцев,игрок,зенит,летом,го,защитник,отдать,аренда,сначала,тульский,арсенал,затем,махачкалинский,анжи,конец,текущий,сезон,право,аренда,ростов,кейра,новосельцев,пожениться,го,настоящее,супруг,футболист,ждать,ребёнок']


In [16]:
pred[0]

'Спорт'

In [19]:
PATH='data/all/'
file =dict_rub[pred[0]]

In [23]:
%%time
df = pd.read_csv(PATH+file+'.csv.bz2',usecols=['date','text'])
df = df[df.text!='text']
df['year'] = df['date'].apply(lambda x: int(x.split(' ')[0].split('-')[0]))
df['month'] = df['date'].apply(lambda x: int(x.split(' ')[0].split('-')[1]))
df = df[df.year>2007]
df['docID'] = list(range(df.shape[0]))
df = df[['docID','year','month','text']]
print(df.head())

       docID  year  month                                               text
14492      0  2008      1  Во вторник 1 января в чемпионате Англии по фут...
14493      1  2008      1  Московский ЦСКА одержал девятую победу в групп...
14494      2  2008      1  Главный тренер мюнхенской "Баварии" Оттмар Хит...
14495      3  2008      1  Корейские врачи приняли решение перестать подд...
14496      4  2008      1  Французская спортивная газета L'Equipe опублик...
CPU times: user 48.9 s, sys: 0 ns, total: 48.9 s
Wall time: 46.4 s


In [47]:
df.index=df['docID']
del df['docID']
df.head()

,year,month,text
docID,,,
0,2008,1,Во вторник 1 января в чемпионате Англии по фут...
1,2008,1,Московский ЦСКА одержал девятую победу в групп...
2,2008,1,"Главный тренер мюнхенской ""Баварии"" Оттмар Хит..."
3,2008,1,Корейские врачи приняли решение перестать подд...
4,2008,1,Французская спортивная газета L'Equipe опублик...


In [9]:
# Загружаем  модель для определения топика
import sys
sys.path.append('/home/midzay/Mlerning/bigartm/python')
import artm

In [10]:
model=artm.load_artm_model(f'/home/midzay/Mlerning/GitHUb/topic_news/bigartm/data/model/bigartm/{dict_rub[pred[0]]}/')

In [11]:
vwpath= f'data/vwpath/999_input_bigartm.vw'
with open(vwpath, 'w') as fp:
    fp.write('| {}\n'.format(' '.join(text)))
    
    # Создаем batches 
batches_path = f'data/batches/999'
if not os.path.exists(batches_path):
    os.makedirs(batches_path)
    
bv = artm.BatchVectorizer(data_path=vwpath,data_format='vowpal_wabbit',target_folder=batches_path)

In [63]:
# Матрица для  нового текста
theta = model.transform(batch_vectorizer=bv)
theta.T

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,0.295066,0.0,0.0,0.0,0.0,0.03434,0.202409,0.146486,0.015665,0.0,0.0,0.0,0.096739,0.008997,0.200297


In [65]:
# Матрица модели для предсказания похожих новостей
theta_join =model.get_theta().T.reset_index()
del theta_join['index']
theta_join.head()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,0.217238,0.725193,0.000000,0.026423,0.0,0.000000,0.000000,0.000000,0.031146,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,0.243041,0.000000,0.000000,0.607071,0.0,0.000000,0.000000,0.104451,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.045437
2,0.000000,0.526068,0.088450,0.000000,0.0,0.000000,0.000000,0.009474,0.008730,0.000000,0.013223,0.000000,0.0,0.000000,0.354055
3,0.000000,0.000000,0.000000,0.000000,0.0,0.006182,0.360268,0.000000,0.123552,0.000000,0.485987,0.024011,0.0,0.000000,0.000000
4,0.000000,0.266688,0.061768,0.016533,0.0,0.000000,0.000000,0.023048,0.000000,0.106447,0.084070,0.235069,0.0,0.206376,0.000000


In [66]:
# Соединяем  текст и theta
joined = df.join(theta_join)
joined.head(2)

,year,month,text,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
docID,,,,,,,,,,,,,,,,,,
0,2008,1,Во вторник 1 января в чемпионате Англии по фут...,0.217238,0.725193,0.0,0.026423,0.0,0.0,0.0,0.000000,0.031146,0.0,0.0,0.0,0.0,0.0,0.000000
1,2008,1,Московский ЦСКА одержал девятую победу в групп...,0.243041,0.000000,0.0,0.607071,0.0,0.0,0.0,0.104451,0.000000,0.0,0.0,0.0,0.0,0.0,0.045437


In [67]:
joined.tail(2)

,year,month,text,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
docID,,,,,,,,,,,,,,,,,,
89084,2019,1,- Биатлонистка сборной России Виктория Сливко...,0.231905,0.166011,0.211924,0.0,0.0,0.330723,0.0,0.059436,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
89085,2019,1,- Двукратный чемпион США по фигурному катанию...,0.218554,0.000000,0.126796,0.0,0.0,0.000000,0.0,0.339773,0.0,0.0,0.0,0.013006,0.0,0.0,0.301871


In [68]:
# Самый ближайший топик для нашей новой новости
max_col =  theta.idxmax()[0]
phi = model.get_phi()
phi['word'] = phi.index
phi['word']=phi.word.apply(lambda x: x[1])

print(f"{max_col}: {phi[[max_col, 'word']].sort_values(by=max_col, ascending=False)['word'].values[:15]}")

topic_0: ['зенит' 'спартак' 'цска' 'локомотив' 'динамо' 'премьерлига' 'рубин'
 'анжи' 'го' 'полузащитник' 'уефа' 'краснодар' 'ростов' 'пол' 'таблица']


In [69]:
# Пять ближайших  топиков для новой новости
cols = theta[0].sort_values(ascending=False).index[:5]
phi = model.get_phi()
phi['word'] = phi.index
phi['word']=phi.word.apply(lambda x: x[1])

for col in cols:
    if col != 'word':
        print(f"{col}: {phi[[col, 'word']].sort_values(by=col, ascending=False)['word'].values[:10]}")

topic_0: ['зенит' 'спартак' 'цска' 'локомотив' 'динамо' 'премьерлига' 'рубин'
 'анжи' 'го' 'полузащитник']
topic_6: ['болельщик' 'фанат' 'произойти' 'инцидент' 'тысяча' 'задержать' 'врач'
 'травма' 'больница' 'драка']
topic_14: ['руководство' 'пост' 'подписать' 'новый' 'соглашение' 'специалист' 'цска'
 'травма' 'наставник' 'информация']
topic_7: ['очень' 'хотеть' 'сделать' 'приводить' 'нужно' 'добавить' 'думать'
 'просто' 'момент' 'болельщик']
topic_12: ['комитет' 'антидопинговый' 'допинг' 'агентство' 'рфс' 'запретить'
 'мокнуть' 'спортивный' 'олимпиада' 'дисквалификация']


In [85]:
# Сортируем документы по  ближайшему топику к нашей новости
joined.sort_values(by=max_col,ascending=False).head(5)

,year,month,text,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
docID,,,,,,,,,,,,,,,,,,
8216,2010,1,"Полузащитник итальянского футбольного клуба ""Б...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16875,2011,9,Новым главным тренером миланского футбольного ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40146,2017,12,Журналист немецкого телеканала ARD Хайо Зеппел...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4489,2009,3,В первой встрече 1/8 финала Кубка Дэвиса между...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87493,2018,7,"- Клуб \""Тампа-Бэй\"" выступающий в Национальн...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
a = joined.sort_values(by=max_col,ascending=False)['text'][:10].values

In [86]:
# Выводим на печать 10 ближайших документов
for i in range (a.shape[0]):
    print (a[i]+'\n')

Полузащитник итальянского футбольного клуба "Брешия" Франческо Флаки во второй раз в своей карьере попался на употреблении кокаина. Положительный результат дала допинг-проба, взятая у игрока 19 декабря 2009 года после матча Серии B (второй по значимости дивизион чемпионата Италии) с "Моденой". Об этом сообщает Associated Press со ссылкой на Олимпийский комитет Италии. Впервые Флаки был уличен в употреблении наркотика в январе 2007 года. Тогда полузащитник выступал за "Сампдорию" в Серии A. Полузащитник был дисквалифицирован на два года. После дисквалификации Флаки возобновил карьеру. В сезоне-2008/09 он выступал за "Эмполи", а летом 2009 года перешел в "Брешию". В чемпионате Серии B сезона-2009/10 Флаки провел 12 матчей и забил один гол. Повторное нарушение антидопинговых правил может привести к окончанию карьеры 34-летнего Флаки. Полузащитник может получить длительную (в том числе - пожизненную) дисквалификацию.

Новым главным тренером миланского футбольного клуба "Интер" стал Клаудио

Косинусное расстояние

In [92]:
import scipy.spatial.distance

In [103]:
u = theta_join.values
u1 = theta_join[:1].values
u1 = np.ravel(u1)

In [163]:
theta_cos = theta.T
v = theta_cos.values
v.shape
v = np.ravel(v)

(1, 15)

In [139]:
import numpy as np
from numpy.linalg import norm
cos_dot = np.dot(v, u1) / norm(u1) / norm(v)
1-cos_dot

0.8097283989191055

In [164]:
l=[]
for i in range (u.shape[0]):
    cos = scipy.spatial.distance.cosine(v,u[i])
    l.append((cos,i))
    

In [165]:
l[:10]

[(0.8097283989191055, 0),
 (0.6765939295291901, 1),
 (0.7474080622196198, 2),
 (0.7286798655986786, 3),
 (0.9733261186629534, 4),
 (0.8513165265321732, 5),
 (1.0, 6),
 (0.6168305277824402, 7),
 (0.9880406986922026, 8),
 (0.993451445363462, 9)]

In [153]:
joined[89061:89062]

,year,month,text,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
docID,,,,,,,,,,,,,,,,,,
89061,2019,1,- Российская теннисистка Дарья Касаткина усту...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
joined[89061:89062].text.values.tolist()[0][:300]

' - Российская теннисистка Дарья Касаткина уступила представительнице Швейцарии Тимеа Бачински в матче первого круга турнира серии Большого шлема Australian Open.\\nВстреча завершилась со счетом 6:3 6:0 в пользу Бачински.\\nСпорт\\n14 января 2019\\nШарапова вышла во второй круг Australian Open обыграв бр'

In [ ]:
'екатерина_гордон,жена,футболист,зенит,иван,новосельцев,катерина,кейра,претензия,клиентка,муж,
приводить,sport,гордон,кейра,пытаться,урегулировать,вопрос,имущественный,характер,господин,новосельцев,позволить,
довольно,нечестный,поведение,достояние,супруга,вопрос,урегулировать,громкий,распря,глушаков,добавить,гордон,пока,
неясно,проводить,новосельцев,манипуляция,недвижимость,чьемуть,совет,футболист,виновный,случиться,ждать,судебный,
разбирательство,новосельцев,игрок,зенит,летом,го,защитник,отдать,аренда,сначала,тульский,арсенал,затем,
махачкалинский,анжи,конец,
текущий,сезон,право,аренда,ростов,кейра,новосельцев,пожениться,го,настоящее,супруг,футболист,ждать,ребёнок'

In [151]:
sorted(l,reverse=True)

[(1.0, 89061),
 (1.0, 89060),
 (1.0, 89059),
 (1.0, 89058),
 (1.0, 89057),
 (1.0, 89040),
 (1.0, 89036),
 (1.0, 89011),
 (1.0, 88982),
 (1.0, 88970),
 (1.0, 88913),
 (1.0, 88911),
 (1.0, 88897),
 (1.0, 88881),
 (1.0, 88837),
 (1.0, 88834),
 (1.0, 88826),
 (1.0, 88801),
 (1.0, 88798),
 (1.0, 88791),
 (1.0, 88775),
 (1.0, 88737),
 (1.0, 88721),
 (1.0, 88701),
 (1.0, 88690),
 (1.0, 88684),
 (1.0, 88618),
 (1.0, 88616),
 (1.0, 88609),
 (1.0, 88580),
 (1.0, 88537),
 (1.0, 88383),
 (1.0, 88377),
 (1.0, 88373),
 (1.0, 88359),
 (1.0, 88340),
 (1.0, 88334),
 (1.0, 88329),
 (1.0, 88281),
 (1.0, 88275),
 (1.0, 88263),
 (1.0, 88252),
 (1.0, 88241),
 (1.0, 88240),
 (1.0, 88238),
 (1.0, 88226),
 (1.0, 88191),
 (1.0, 88190),
 (1.0, 88180),
 (1.0, 88156),
 (1.0, 88127),
 (1.0, 88014),
 (1.0, 87990),
 (1.0, 87962),
 (1.0, 87946),
 (1.0, 87776),
 (1.0, 87733),
 (1.0, 87723),
 (1.0, 87680),
 (1.0, 87634),
 (1.0, 87626),
 (1.0, 87623),
 (1.0, 87613),
 (1.0, 87587),
 (1.0, 87564),
 (1.0, 87554),
 (1.0, 875

In [104]:
u.shape

(89086, 15)